In [1]:
import tensorflow as tf 
from tensorflow.keras import datasets,layers,models

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
tf.__version__

'2.2.0'

In [2]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import models,layers,preprocessing,optimizers,losses,metrics
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re,string

In [3]:
tf.print('c')

c


In [5]:
a = tf.constant('100')
tf.print(a)


100


In [6]:
train_data_path = "./data/imdb/train.csv"
test_data_path =  "./data/imdb/test.csv"

In [7]:
MAX_WORDS = 10000  # 仅考虑最高频的10000个词
MAX_LEN = 200  # 每个样本保留200个词的长度
BATCH_SIZE = 20 

In [43]:
#构建管道
def split_line(line):
    arr = tf.strings.split(line,"\t")
    label = tf.expand_dims(tf.cast(tf.strings.to_number(arr[0]),tf.int32),axis = 0)
    text = tf.expand_dims(arr[1],axis = 0)
    return (text,label)

In [44]:
ds_train_raw =  tf.data.TextLineDataset(filenames = [train_data_path]) \
   .map(split_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .shuffle(buffer_size = 1000) \
   .batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

In [35]:
ds_test_raw = tf.data.TextLineDataset(filenames = [test_data_path]) \
   .map(split_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

Tensor("strided_slice:0", shape=(), dtype=string)


In [65]:
pdf_train = pd.read_table('./data/imdb/train.csv', header=None)
pdf_train.head()

,0,1
0,0,It really boggles my mind when someone comes a...
1,0,Mary Pickford becomes the chieftain of a Scott...
2,0,"Well, at least my theater group did, lol. So o..."
3,1,I must give How She Move a near-perfect rating...
4,0,"I must say, when I read the storyline on the b..."


In [62]:
pdf_test = pd.read_table('./data/imdb/test.csv', header=None)
pdf_test.head()

,0,1
0,1,The first one meant victory. This one means de...
1,1,"Excellent movie, a realistic picture of contem..."
2,1,This film is moving without being sentimental ...
3,0,This is high grade cheese fare of B movie kung...
4,1,Halloween is the story of a boy who was misund...


In [77]:
#构建词典
def clean_text(text):
    lowercase = tf.strings.lower(text)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    cleaned_punctuation = tf.strings.regex_replace(stripped_html,
         '[%s]' % re.escape(string.punctuation),'')
    return cleaned_punctuation

In [78]:
vectorize_layer = TextVectorization(
    standardize=clean_text,
    split = 'whitespace',
    max_tokens=MAX_WORDS-1, #有一个留给占位符
    output_mode='int',
    output_sequence_length=MAX_LEN)

In [81]:
vectorize_layer

In [82]:
type(vectorize_layer)

tensorflow.python.keras.layers.preprocessing.text_vectorization.TextVectorization

In [89]:
vectorize_layer.__dict__

{'_self_setattr_tracking': True,
 '_obj_reference_counts_dict': ObjectIdentityDictionary({<_ObjectIdentityWrapper wrapping 9999>: 1, <_ObjectIdentityWrapper wrapping 2>: 1, <_ObjectIdentityWrapper wrapping 1>: 1, <_ObjectIdentityWrapper wrapping 9998>: 1, <_ObjectIdentityWrapper wrapping <function clean_text at 0x63afcb950>>: 1, <_ObjectIdentityWrapper wrapping 'whitespace'>: 1, <_ObjectIdentityWrapper wrapping 'int'>: 1, <_ObjectIdentityWrapper wrapping 200>: 1, <_ObjectIdentityWrapper wrapping True>: 2, <_ObjectIdentityWrapper wrapping 0>: 1, <_ObjectIdentityWrapper wrapping False>: 1, <_ObjectIdentityWrapper wrapping <_TextVectorizationCombiner>>: 1, <_ObjectIdentityWrapper wrapping DictWrapper(OrderedDict())>: 1, <_ObjectIdentityWrapper wrapping <tensorflow.python.keras.layers.preprocessing.index_lookup.IndexLookup object at 0x63b01c390>>: 1}),
 '_max_tokens': 9999,
 '_reserved_values': 2,
 '_oov_value': 1,
 '_max_vocab_size': 9998,
 '_standardize': <function __main__.clean_text(te

In [83]:
ds_text = ds_train_raw.map(lambda text,label: text)

In [90]:
ds_text

<MapDataset shapes: (None, 1), types: tf.string>

In [86]:
vectorize_layer.adapt(ds_text)

In [88]:
print(vectorize_layer.get_vocabulary()[0:100])

[b'the', b'and', b'a', b'of', b'to', b'is', b'in', b'it', b'i', b'this', b'that', b'was', b'as', b'for', b'with', b'movie', b'but', b'film', b'on', b'not', b'you', b'his', b'are', b'have', b'be', b'he', b'one', b'its', b'at', b'all', b'by', b'an', b'they', b'from', b'who', b'so', b'like', b'her', b'just', b'or', b'about', b'has', b'if', b'out', b'some', b'there', b'what', b'good', b'more', b'when', b'very', b'she', b'even', b'my', b'no', b'would', b'up', b'time', b'only', b'which', b'story', b'really', b'their', b'were', b'had', b'see', b'can', b'me', b'than', b'we', b'much', b'well', b'get', b'been', b'will', b'into', b'people', b'also', b'other', b'do', b'bad', b'because', b'great', b'first', b'how', b'him', b'most', b'dont', b'made', b'then', b'them', b'films', b'movies', b'way', b'make', b'could', b'too', b'any', b'after', b'characters']


In [91]:

#单词编码
ds_train = ds_train_raw.map(lambda text,label:(vectorize_layer(text),label)) \
    .prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test_raw.map(lambda text,label:(vectorize_layer(text),label)) \
    .prefetch(tf.data.experimental.AUTOTUNE)

In [105]:
next(iter(ds_test.unbatch()))

(<tf.Tensor: shape=(200,), dtype=int64, numpy=
 array([   2,   85,   28,  954, 5440,   11,   28,  779, 4206,    9,  295,
         265,    8,    4, 9210,   47,    2,    1,   24, 1148,    3,    1,
           3,   89,  893,   12,   72, 2391,   35,    2, 8146,   44,   22,
         118,  123, 1705,  466,   22,  375,   86,    9,  603,   22,   76,
         375,    9,   54,  200,   12, 1692, 4424, 9478,    7,  270,  170,
        3567,   27,  263,   20, 1416,   11, 4840,   42,    2, 4138,   36,
        1236,    2,  166, 3672,    5,  993,   41, 2122,  177,   48, 9868,
           7, 1685,    6,    2, 1757,   18,  126,   29,   37,   73, 1013,
           2,  670,  529,    7,   47,  183,    2, 4840,   61,    7,   28,
           5,    2,  783, 7710,    8,   11,  195, 5911, 1127,   27,   43,
        4095,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    

In [110]:
next(iter(ds_test.unbatch().take(2)))

(<tf.Tensor: shape=(200,), dtype=int64, numpy=
 array([   2,   85,   28,  954, 5440,   11,   28,  779, 4206,    9,  295,
         265,    8,    4, 9210,   47,    2,    1,   24, 1148,    3,    1,
           3,   89,  893,   12,   72, 2391,   35,    2, 8146,   44,   22,
         118,  123, 1705,  466,   22,  375,   86,    9,  603,   22,   76,
         375,    9,   54,  200,   12, 1692, 4424, 9478,    7,  270,  170,
        3567,   27,  263,   20, 1416,   11, 4840,   42,    2, 4138,   36,
        1236,    2,  166, 3672,    5,  993,   41, 2122,  177,   48, 9868,
           7, 1685,    6,    2, 1757,   18,  126,   29,   37,   73, 1013,
           2,  670,  529,    7,   47,  183,    2, 4840,   61,    7,   28,
           5,    2,  783, 7710,    8,   11,  195, 5911, 1127,   27,   43,
        4095,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    

In [93]:
# 演示自定义模型范例，实际上应该优先使用Sequential或者函数式API
tf.keras.backend.clear_session()

class CnnModel(models.Model):
    def __init__(self):
        super(CnnModel, self).__init__()
        
    def build(self,input_shape):
        self.embedding = layers.Embedding(MAX_WORDS,7,input_length=MAX_LEN)
        self.conv_1 = layers.Conv1D(16, kernel_size= 5,name = "conv_1",activation = "relu")
        self.pool_1 = layers.MaxPool1D(name = "pool_1")
        self.conv_2 = layers.Conv1D(128, kernel_size=2,name = "conv_2",activation = "relu")
        self.pool_2 = layers.MaxPool1D(name = "pool_2")
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(1,activation = "sigmoid")
        super(CnnModel,self).build(input_shape)
    
    def call(self, x):
        x = self.embedding(x)
        x = self.conv_1(x)
        x = self.pool_1(x)
        x = self.conv_2(x)
        x = self.pool_2(x)
        x = self.flatten(x)
        x = self.dense(x)
        return(x)
    
    # 用于显示Output Shape
    def summary(self):
        x_input = layers.Input(shape = MAX_LEN)
        output = self.call(x_input)
        model = tf.keras.Model(inputs = x_input,outputs = output)
        model.summary()
    
model = CnnModel()
model.build(input_shape =(None,MAX_LEN))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 7)            70000     
_________________________________________________________________
conv_1 (Conv1D)              (None, 196, 16)           576       
_________________________________________________________________
pool_1 (MaxPooling1D)        (None, 98, 16)            0         
_________________________________________________________________
conv_2 (Conv1D)              (None, 97, 128)           4224      
_________________________________________________________________
pool_2 (MaxPooling1D)        (None, 48, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 6144)              0     

In [94]:
#打印时间分割线
@tf.function
def printbar():
    today_ts = tf.timestamp()%(24*60*60)
    
    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8+timestring)

  

In [95]:
optimizer = optimizers.Nadam()
loss_func = losses.BinaryCrossentropy()

train_loss = metrics.Mean(name='train_loss')
train_metric = metrics.BinaryAccuracy(name='train_accuracy')

valid_loss = metrics.Mean(name='valid_loss')
valid_metric = metrics.BinaryAccuracy(name='valid_accuracy')

In [96]:
@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features,training = True)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_metric.update_state(labels, predictions)

In [97]:

@tf.function
def valid_step(model, features, labels):
    predictions = model(features,training = False)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)

In [99]:
def train_model(model,ds_train,ds_valid,epochs):
    for epoch in tf.range(1,epochs+1):
        
        for features, labels in ds_train:
            train_step(model,features,labels)

        for features, labels in ds_valid:
            valid_step(model,features,labels)
        
        #此处logs模板需要根据metric具体情况修改
        logs = 'Epoch={},Loss:{},Accuracy:{},Valid Loss:{},Valid Accuracy:{}' 
        
        if epoch%1==0:
            printbar()
            tf.print(tf.strings.format(logs,
            (epoch,train_loss.result(),train_metric.result(),valid_loss.result(),valid_metric.result())))
            tf.print("")
        
        train_loss.reset_states()
        valid_loss.reset_states()
        train_metric.reset_states()
        valid_metric.reset_states()

In [100]:
train_model(model,ds_train,ds_test,epochs = 6)

================================================================================16:41:27
Epoch=1,Loss:0.440837353,Accuracy:0.773,Valid Loss:0.324008226,Valid Accuracy:0.8598

================================================================================16:41:33
Epoch=2,Loss:0.250649959,Accuracy:0.90025,Valid Loss:0.33694756,Valid Accuracy:0.866

================================================================================16:41:40
Epoch=3,Loss:0.181023493,Accuracy:0.93125,Valid Loss:0.371792346,Valid Accuracy:0.8696

================================================================================16:41:46
Epoch=4,Loss:0.128877416,Accuracy:0.9542,Valid Loss:0.440727264,Valid Accuracy:0.8638

================================================================================16:41:53
Epoch=5,Loss:0.0832502171,Accuracy:0.97155,Valid Loss:0.578641713,Valid Accuracy:0.8572

================================================================================16:42:00
Epoch=6,Loss:0.0487264283,Accu

In [101]:
def evaluate_model(model,ds_valid):
    for features, labels in ds_valid:
         valid_step(model,features,labels)
    logs = 'Valid Loss:{},Valid Accuracy:{}' 
    tf.print(tf.strings.format(logs,(valid_loss.result(),valid_metric.result())))
    
    valid_loss.reset_states()
    train_metric.reset_states()
    valid_metric.reset_states()

In [102]:
evaluate_model(model,ds_test)

Valid Loss:0.723280847,Valid Accuracy:0.8564
